In [11]:
import email
from email import policy
from email.parser import BytesParser
from pathlib import Path
import os

In [32]:
eml_file_path = 'C:/Users/user/Downloads/[피피에스] 사업 공고 전달의 건.eml'
pdf_file_path = './mail_test.pdf'

In [34]:
# .eml 파일 읽기
with open(eml_file_path, 'rb') as eml_file:
    msg = BytesParser(policy=policy.default).parse(eml_file)
print(type(msg))
print([x for x,y in msg.items()])

<class 'email.message.EmailMessage'>
['X-Priority', 'Importance', 'Message-ID', 'Date', 'From', 'Reply-To', 'To', 'Subject', 'Cc', 'Mime-Version', 'Content-Type']


In [131]:
for x in msg.items():
    print(x)

('X-Priority', '3')
('Importance', 'normal')
('Message-ID', '<1243321577.1718269786794.JavaMail.root@localhost>')
('Date', 'Thu, 13 Jun 2024 18:09:46 +0900')
('From', '김나래 <narae3759@ppsystem.co.kr>')
('Reply-To', '김나래 <narae3759@ppsystem.co.kr>')
('To', 'hi@bitstep.it')
('Subject', '[피피에스] 사업 공고 전달의 건')
('Cc', '박석준 <parksj@ppsystem.co.kr>')
('Mime-Version', '1.0')
('Content-Type', 'multipart/mixed; boundary="----=_Part_2350126_834655295.1718269786794"')


In [35]:
for x in msg.walk():
    content_type = x.get_content_type()
    print(f"Content-Type: {content_type}")

for x in msg.walk():
    content_type = x.get_content_type()
    isfile = x.get('Content-disposition')
    print(f"Content-Type: {content_type}, 첨부파일: {isfile}")

Content-Type: multipart/mixed
Content-Type: multipart/related
Content-Type: multipart/alternative
Content-Type: text/plain
Content-Type: text/html
Content-Type: image/png
Content-Type: application/hwp
Content-Type: multipart/mixed, 첨부파일: None
Content-Type: multipart/related, 첨부파일: None
Content-Type: multipart/alternative, 첨부파일: None
Content-Type: text/plain, 첨부파일: None
Content-Type: text/html, 첨부파일: None
Content-Type: image/png, 첨부파일: inline; filename="clipboardImage.png"
Content-Type: application/hwp, 첨부파일: attachment; filename="제안요청서_AI기반 현안 해결형 데이터 분석모델_최종.hwp"


In [38]:
# 본문 내용 추출
html_content = ""
for part in msg.walk():
    content_type = part.get_content_type()
    if content_type == "text/html":
        try:
            html_content = part.get_payload(decode=True).decode()
        except:
            html_content = part.get_payload(decode=True).decode("CP949")

print(html_content)

<div class="TerraceMsg">
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">이서준 대표님 안녕하세요. 피피에스의 김나래 연구원입니다.&nbsp;</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">오늘 공유된&nbsp;사업 공고를 전달드립니다.&nbsp;</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">---</p>
 <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">- <b>사업 공고명</b>: <a href="https://w

In [41]:
# 이미지 추출
img_path = Path("./")
img_sources = {}

for part in msg.walk():
    content_type = part.get_content_type()
    if content_type.startswith("image"):
        cid = part.get("Content-ID").strip("<>")
        filename = part.get_filename()
        if not cid: continue # 본문에 이미지가 있는 경우에만 `Content-ID`가 있다. 

        file_path = img_path / part.get_filename()
        img_sources[cid] = file_path
        with open(file_path, 'wb') as img_file:
            img_file.write(part.get_payload(decode=True))

0128c005-9dfe-4d36-876b-4eec8d378654


In [42]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_content)
img_tags = soup.find_all("img")
for img in img_tags:
    cid = img["src"].lstrip("cid:")
    img["src"] = img_sources[cid]
html_content = str(soup)

In [106]:
import email

attached_files = []
for part in msg.walk():
    disposition = part.get_content_disposition()
    if disposition=="attachment":
        print(disposition)
        filename = part.get_filename()
        attached_files.append(filename)

print(attached_files)

attachment
['제안요청서_AI기반 현안 해결형 데이터 분석모델_최종.hwp']


In [103]:
part.get_content_disposition()

'attachment'

In [113]:
# eml2html 최종 코드

img_path = Path("./")

html_content = ""
img_sources = {}
attached_files = []

# .eml 파일 읽기
with open(eml_file_path, 'rb') as eml_file:
    msg = BytesParser(policy=policy.default).parse(eml_file)

# 추출
for part in msg.walk():
    content_type = part.get_content_type()
    disposition = part.get_content_disposition()
    filename = part.get_filename()
    if content_type == "text/html":
        try:
            html_content = part.get_payload(decode=True).decode()
        except:
            html_content = part.get_payload(decode=True).decode("CP949")

    if disposition == "inline":
        cid = part.get("Content-ID").strip("<>")

        file_path = img_path / part.get_filename()
        with open(file_path, 'wb') as img_file:
            img_file.write(part.get_payload(decode=True))
        
        img_sources[cid] = file_path.resolve()
    elif disposition == "attachment":
        attached_files.append(filename)

soup = BeautifulSoup(html_content)
img_tags = soup.find_all("img")
for img in img_tags:
    cid = img["src"].lstrip("cid:")
    img["src"] = img_sources[cid]
html_content = str(soup)

print(html_content)


<html><body><div class="TerraceMsg">
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>이서준 대표님 안녕하세요. 피피에스의 김나래 연구원입니다. </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>오늘 공유된 사업 공고를 전달드립니다. </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>---</p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>- <b>사업 공고명</b>: <a href="https://www.daehanbid.co.kr/common/mail_bid.htm?ukey=ae67f4c79cb6324f17f6a61d5a1b1a46&amp;BidNo=20240610

In [114]:
# Headers 추출하기
header_list = ["Subject", "To", "From", "Cc", "BCc", "Date", "Attached_Files"]
headers = {}
for key in header_list:
    if key in msg:
        headers[key] = msg[key].replace('<', '&lt;').replace('>', '&gt;')
    else:
        headers[key] = ""
headers["Attached_Files"] = '\n'.join(attached_files)
print(headers)

{'Subject': '[피피에스] 사업 공고 전달의 건', 'To': 'hi@bitstep.it', 'From': '김나래 &lt;narae3759@ppsystem.co.kr&gt;', 'Cc': '박석준 &lt;parksj@ppsystem.co.kr&gt;', 'BCc': '', 'Date': 'Thu, 13 Jun 2024 18:09:46 +0900', 'Attached_Files': '제안요청서_AI기반 현안 해결형 데이터 분석모델_최종.hwp'}


In [132]:
test = '박석준 <parksj@ppsystem.co.kr>'
test.replace('<', '&lt;').replace('>', '&gt;')

'박석준 &lt;parksj@ppsystem.co.kr&gt;'

In [133]:
import html 

text = "김나래 <narae3759@gmail.co.kr>"
print(html.escape(text))

김나래 &lt;narae3759@gmail.co.kr&gt;


In [124]:
html_header = f"""
<div class="header">
    <p style='font-size:24px;'><strong>제목:</strong> {headers['Subject']}</p>
    <p><strong>보낸사람:</strong> {headers['From']}</p>
    <p><strong>받는사람:</strong> {headers['To']}</p>
    <p><strong>참조:</strong> {headers['Cc']}</p>
    <p><strong>숨은참조:</strong> {headers['BCc']}</p>
    <p><strong>보낸날짜:</strong> {headers['Date']}</p>
    <p><strong>첨부파일:</strong> {headers['Attached_Files']}</p>
</div>
"""

In [128]:
html = f"""
<html>
<head>
    <meta charset="UTF-8">
    <style>
        body {{ font-family: 맑은 고딕; }}
        .header {{ font-size: 13px; }}
    </style>
</head>
<body>
    <hr>
    {html_header}
    <hr>    
    {html_content}
</body>
</html>
"""

In [129]:
print(html)


<html>
<head>
    <meta charset="UTF-8">
    <style>
        body { font-family: 맑은 고딕; }
        .header { font-size: 13px; }
    </style>
</head>
<body>
    <hr>
    
<div class="header">
    <p style='font-size:24px;'><strong>제목:</strong> [피피에스] 사업 공고 전달의 건</p>
    <p><strong>보낸사람:</strong> 김나래 &lt;narae3759@ppsystem.co.kr&gt;</p>
    <p><strong>받는사람:</strong> hi@bitstep.it</p>
    <p><strong>참조:</strong> 박석준 &lt;parksj@ppsystem.co.kr&gt;</p>
    <p><strong>숨은참조:</strong> </p>
    <p><strong>보낸날짜:</strong> Thu, 13 Jun 2024 18:09:46 +0900</p>
    <p><strong>첨부파일:</strong> 제안요청서_AI기반 현안 해결형 데이터 분석모델_최종.hwp</p>
</div>

    <hr>    
    <html><body><div class="TerraceMsg">
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>이서준 대표님 안녕하세요. 피피에스의 김나래 연구원입니다. </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 

In [117]:
import pdfkit

config = pdfkit.configuration(wkhtmltopdf="C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe")
options = {
    "encoding": "UTF-8",
    "enable-local-file-access": None
}

In [130]:
pdfkit.from_string(html, 'test.pdf', configuration=config, options=options)

True

In [39]:
headers['Cc']

'박석준 &lt;parksj@ppsystem.co.kr&gt;'

In [70]:
html_header = f"""/
<div class="header">
    <p style='font-size:24px;'><strong>제목:</strong> {headers['Subject']}</p>
    <p><strong>보낸사람:</strong> {headers['From']}</p>
    <p><strong>받는사람:</strong> {headers['To']}</p>
    <p><strong>참조:</strong> {headers['Cc']}</p>
    <p><strong>숨은참조:</strong> {headers['BCc']}</p>
    <p><strong>보낸날짜:</strong> {headers['Date']}</p>
</div>
"""

In [29]:
html_content

'<div class="TerraceMsg">\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">이서준 대표님 안녕하세요. 피피에스의 김나래 연구원입니다.&nbsp;</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">오늘 공유된&nbsp;사업 공고를 전달드립니다.&nbsp;</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">&nbsp;</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">---</p>\n <p style="font-family: &quot;맑은 고딕&quot;; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;">- <b>사업 공고명</b>: <a href="h

In [100]:
html = f"""/
<html>
<head>
    <meta charset="UTF-8">
    <style>
        body {{ font-family: 맑은 고딕; }}
        .header {{ font-size: 13px; }}
    </style>
</head>
<body>
    <hr>
    {html_header}
    <hr>    
    {html_content}
</body>
</html>
"""

In [101]:
print(html)

/
<html>
<head>
    <meta charset="UTF-8">
    <style>
        body { font-family: 맑은 고딕; }
        .header { font-size: 13px; }
    </style>
</head>
<body>
    <hr>
    <div class="header">
    <p style='font-size:24px;'><strong>제목:</strong> [피피에스] 사업 공고 전달의 건</p>
    <p><strong>보낸사람:</strong> 김나래 &lt;narae3759@ppsystem.co.kr&gt;</p>
    <p><strong>받는사람:</strong> hi@bitstep.it</p>
    <p><strong>참조:</strong> 박석준 &lt;parksj@ppsystem.co.kr&gt;</p>
    <p><strong>숨은참조:</strong> </p>
    <p><strong>보낸날짜:</strong> Thu, 13 Jun 2024 18:09:46 +0900</p>
</div>

    <hr>    
    <html><body><div class="TerraceMsg">
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'>이서준 대표님 안녕하세요. 피피에스의 김나래 연구원입니다. </p>
<p style='font-family: "맑은 고딕"; font-size: 10pt; line-height: 150%; margin-top: 0px; margin-bottom: 0px;'> </p>
<p style='font-fam

In [85]:
import pdfkit

config = pdfkit.configuration(wkhtmltopdf="C:/Program Files/wkhtmltopdf/bin/wkhtmltopdf.exe")
options = {
    "encoding": "UTF-8",
    "enable-local-file-access": None
}

In [102]:
pdfkit.from_string(html, 'test.pdf', configuration=config, options=options)

True

In [1]:
from pathlib import Path 

path = Path("C:/Program Files/wkhtmltopdf/bin/wkhtml.topdf.exe")
print(path.name)
print(path.parent.stem)
print(path.stem)

wkhtml.topdf.exe
bin
wkhtml.topdf


In [142]:
path = Path("./eml_folder")
files = path.glob("*.eml")
for file in files:
    print(file)

eml_folder\[피피에스] 3주차 과제 안내.eml
eml_folder\[피피에스] API Key 전달의 건.eml
eml_folder\[피피에스] Pandas 관련 실습 자료.eml
eml_folder\[피피에스] Pandas 관련 참고 자료.eml
eml_folder\[피피에스] url 추가.eml
eml_folder\[피피에스] 관련 교육 자료.eml
eml_folder\[피피에스] 교육 관련 자료(240530).eml
eml_folder\[피피에스] 교육 관련 자료(240531).eml
eml_folder\[피피에스] 안녕하세요. 김나래 연구원입니다..eml
eml_folder\[피피에스] 인턴 교육 안내(240603).eml
eml_folder\[피피에스] 인턴 교육 안내(240604).eml
eml_folder\[피피에스] 인턴 교육 안내(240605).eml
eml_folder\[피피에스] 인턴 교육 안내(240610).eml
eml_folder\[피피에스] 인턴 교육 안내(240612).eml
eml_folder\[피피에스] 인턴 교육 안내(240613).eml
eml_folder\[피피에스] 인턴 교육 안내(240617).eml
eml_folder\[피피에스] 인턴 교육 안내(240620).eml
eml_folder\[피피에스] 인턴 교육 안내(240621).eml


In [ ]:
import os
os.path.isdir(path.parent)

True

In [ ]:
import logging

logger = logging.getLogger(__name__)

try:
    1 / 0
except ZeroDivisionError as e:
    logger.error("Error occurred")

Error occurred


In [ ]:
import logging

# 로거 설정
logger = logging.getLogger('example_logger')
logger.setLevel(logging.DEBUG)  # 로거의 레벨을 DEBUG로 설정

# 콘솔 핸들러와 포맷 설정
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)  # 콘솔 핸들러의 레벨을 DEBUG로 설정
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

# 디버그 메시지 출력
logger.debug("This is a debug message.")
logger.info("This is an info message.")
logger.warning("This is a warning message.")
logger.error("This is an error message.")
logger.critical("This is a critical message.")


2024-06-20 17:29:00,595 - example_logger - DEBUG - This is a debug message.
2024-06-20 17:29:00,596 - example_logger - INFO - This is an info message.
2024-06-20 17:29:00,597 - example_logger - WARNING - This is a warning message.
2024-06-20 17:29:00,598 - example_logger - ERROR - This is an error message.
2024-06-20 17:29:00,599 - example_logger - CRITICAL - This is a critical message.


In [13]:
from datetime import datetime

test = "Tue, 04 Jun 2024 09:38:52 +0900"
format="%a, %d %b %Y %H:%M:%S %z"
datetime.strptime(test,format).strftime(format="(%y%m%d)")

'(240604)'

In [5]:
gen1 = (x for x in [])
gen2 = (x for x in [1, 2, 3])
print(type(gen1), type(gen2))

next(gen1)

<class 'generator'> <class 'generator'>


StopIteration: 